# **Smart Persistence Model (SPM) Implementation**



In the following i will provide **two implementations** for the benchmarck model.

**First** my own implementation, this implemntation usese the PVLib libary to calculate the sun angles at any given time.

**Second** the implementation by **Yuchao Nie** witch was used in the SUNSET forcast model. (Source: https://github.com/yuhao-nie/Stanford-solar-forecasting-dataset)
 Code was Open-sourced using the MIT License:
 Copyright (c) 2022 Yuhao Nie, Xiatong Li 





## Imports and Defines

In [ ]:
import numpy as np
import pandas as pd
!pip install pvlib
import pvlib
import matplotlib.pyplot as plt
import os
import pickle
from math import *
import calendar
import datetime
import time

from spm_functions import *

In [2]:
# Def. functions

def root_mean_squared_error(y_true, y_pred):
    """
    Root Mean Squared Error (RMSE) custom metric function.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mean_absolute_error(y_true, y_pred):
    """
    Mean Absolute Error (MAE) custom metric function.
    """
    return np.mean(np.abs(y_pred - y_true))

## 1. How to use my Implemantation



In [8]:
# Def. parameters


# Standart parameters for the SKIPPD Dataset (Source: https://doi.org/10.1016/j.solener.2023.03.043)
parameters = {
            'latitude': 37.42808,           # Latitude of the location in decimal degrees
            'longitude': -122.17023,        # Longitude of the location in decimal degrees
            'altitude': 23,                 # Altitude of the location in meters
            'panel_elevation': 22.5 ,       # Elevation angle of the solar PV arrays in degrees
            'panel_azimuth': 195,           # Azimuth angle of the solar PV arrays in degrees
            'max_solar_irradiance': 1000,   # Maximum solar irradiance in W/m^2
            'effective_panel_area': 24.98,  # Effective PV panel area in square meters
            'time_delta': 15,               # forecast horizon in minutes
            'time_zone_center_longitude': -120    # time zone correction, for youre time zone, behind or before UTC (Coordinated Universal Time).
        }


In [15]:
start_time = time.time()


# Function call:
predictions_implementation_1 = spm_pv_forecast_v1(data_set["labels_test"], data_set["datetime_labels_test"],  parameters=parameters)


end_time = time.time()


mae_implementation_1 = round(mean_absolute_error(data_set["labels_test"][15:], predictions_implementation_1[:-15]), 4)
execution_time_implementation_1 = round(end_time - start_time, 2)
rmse_implementation_1 = round(root_mean_squared_error(data_set["labels_test"][15:], predictions_implementation_1[:-15]), 4)

print(f"The MAE of the SPM is: {mae_implementation_1} kW")
print(f"The RMSE of the SPM is: {rmse_implementation_1} kW")
print(f"Execution time: {execution_time_implementation_1} seconds")


The MAE of the SPM is: 2.0394 kW
The RMSE of the SPM is: 3.6559 kW
Execution time: 91.52 seconds


## 2. How to use the implementation by Yuchao Nie (SKIPP'D)

This implementation requires adjustments for use with a different dataset.

Source:
[SKIPP´D Dataset](https://github.com/yuhao-nie/Stanford-solar-forecasting-dataset)


In [10]:
# Def. parameters


# Standart parameters for the SKIPPD Dataset (Source: https://doi.org/10.1016/j.solener.2023.03.043)
parameters = {
            'latitude': 37.42808,           # Latitude of the location in decimal degrees
            'longitude': -122.17023,        # Longitude of the location in decimal degrees
            'altitude': 23,                 # Altitude of the location in meters
            'panel_elevation': 22.5 ,       # Elevation angle of the solar PV arrays in degrees
            'panel_azimuth': 195,           # Azimuth angle of the solar PV arrays in degrees
            'max_solar_irradiance': 1000,   # Maximum solar irradiance in W/m^2
            'effective_panel_area': 24.98,  # Effective PV panel area in square meters
            'time_delta': 15,               # forecast horizon in minutes
            'time_zone_center_longitude': -120    # time zone correction, for youre time zone, behind or before UTC (Coordinated Universal Time).
        }


In [16]:
custom_time_delta = 15  # int number for the forcast piriod in minutes


start_time = time.time()


# Function call:
predictions_implementation_2 = spm_pv_forecast_v2(data_set["labels_test"], data_set["datetime_labels_test"],  parameters=parameters)



end_time = time.time()


mae_implementation_2 = round(mean_absolute_error(data_set["labels_test"][15:], predictions_implementation_2[:-15]), 4)
execution_time_implementation_2 = round(end_time - start_time, 2)
rmse_implementation_2 = round(root_mean_squared_error(data_set["labels_test"][15:], predictions_implementation_2[:-15]), 4)

print(f"The MAE of the SPM is: {mae_implementation_2} kW")
print(f"The RMSE of the SPM is: {rmse_implementation_2} kW")
print(f"Execution time: {execution_time_implementation_2} seconds")


The MAE of the SPM is: 1.6711 kW
The RMSE of the SPM is: 3.5654 kW
Execution time: 0.66 seconds


# Comparison of the two SPM implementations

Below, I will provide some statistics comparing the two implementations so you can hopefully choose the one most suited for you.

I utilized the test set of the SKIPP'D Benchmark Dataset to acquire these metrics. ([source](https://github.com/yuhao-nie/Stanford-solar-forecasting-dataset) )


The ***main difference*** between the two implementations is how the zenith and azimuth angles of the sun are calculated.

My implementation utilizes the NREL SPA algorithm through the PVlib library, while the one from Yuchao Nie uses estimations based on empirical functions from da Rosa (Source 5).


In [17]:
# Testet with 13973 Datapoints (SKIPPD Testset)

print("Comparison of Metrics for Two Models:")
print("-" * 50)
print(f"{'Metric':<20} {'implementation 1':<20} {'implementation 2':<20}")
print("-" * 50)
print(f"{'RMSE (kW)':<20} {rmse_implementation_1:<20} {rmse_implementation_2:<20}")
print(f"{'MAE (kW)':<20} {mae_implementation_1:<20} {mae_implementation_2:<20}")
print(f"{'Execution Time (s)':<20} {execution_time_implementation_1:<20} {execution_time_implementation_2:<20}")
print("-" * 50)



Comparison of Metrics for Two Models:
--------------------------------------------------
Metric               implementation 1     implementation 2    
--------------------------------------------------
RMSE (kW)            3.6559               3.5654              
MAE (kW)             2.0394               1.6711              
Execution Time (s)   91.52                0.66                
--------------------------------------------------
